In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
TEMP_PT_EPIC_concat = dataiku.Dataset("TEMP_PT_EPIC_concat")
df = TEMP_PT_EPIC_concat.get_dataframe()
df.head()




In [0]:
from sklearn.base import BaseEstimator,TransformerMixin
import numpy as np
import pandas as pd

In [0]:
X=df.drop(['IsNonPromoter','ISPROMOTER','EncounterKey','EncounterEpicCsn','PrimaryMrn'],axis=1)
y=df['IsNonPromoter']

In [0]:
cat_cols=X.select_dtypes(include='object').columns

epsilon=1e-4
total_events=y.sum()
total_non_events=len(y)-total_events

In [0]:
cat_transformed=pd.DataFrame()

In [0]:
# https://www.analyticsvidhya.com/blog/2021/06/understand-weight-of-evidence-and-information-value/
# https://medium.com/@mailtoanupammaiti/weight-of-evidence-woe-and-information-value-iv-893635d1423c

for col in cat_cols:
    woe_mapping={}
    for category in X[col].unique():
        mask=(X[col]==category)
        events=y[mask].sum()
        non_events=mask.sum()-events
        event_rate=events/total_events if total_events>0 else epsilon
        non_event_rate=non_events/total_non_events if total_non_events>0 else epsilon
        woe_value=np.log((non_event_rate+epsilon)/(event_rate+epsilon))
        woe_mapping[category]=woe_value
    cat_transformed[col+'_woe']=X[col].map(woe_mapping)

In [0]:
cat_transformed

In [0]:
num_cols=X.select_dtypes(include='number').columns
num_transformed=pd.DataFrame()

In [0]:
for col in num_cols:
    unique_vals=X[col].unique()
    
    if len(unique_vals)==2:
        norm=X[col]
    else:
     col_min=X[col].min()
     col_max=X[col].max()
     range_val=col_max-col_min
     norm=(X[col]-col_min)/range_val if range_val!=0 else 0
    num_transformed[col+'_norm_weight']=norm

In [0]:
num_transformed

In [0]:
final_df=pd.concat([cat_transformed,num_transformed],axis=1)
final_df['IsNonPromoter']=df['IsNonPromoter']
final_df.head()

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

TEMP_PT_EPIC_concat_w_df = df # For this sample code, simply copy input to output


# Write recipe outputs
TEMP_PT_EPIC_concat_w = dataiku.Dataset("TEMP_PT_EPIC_concat_w")
TEMP_PT_EPIC_concat_w.write_with_schema(final_df)